## 모델 출력 확인하기
1. 모델 학습/평가 관리용 클래스인 ModelManager의 인스턴스를 생성합니다.
2. 학습/평가용으로 불러올 전처리 데이터의 구조는 `BaseDataset` 클래스로 구현됩니다.
3. `BaseDataset`은 `torch.utils.data.Dataset` 클래스를 상속받는데, 이를 통해 전처리 데이터를 `DataLoader`로 관리할 수 있게 됩니다.
4. `DataLoader`는 `BaseDataset`에 구현된 데이터 구조를 batch data 형태로 바꾸기 위해 데이터의 모든 요소에 차원을 하나 추가합니다.<br/>
   -> 상세: `BaseDataset`은 인스턴스 생성 시(즉 __init__함수에서) `list[dict[str, Tensor]]` 형태의 데이터를 생성하고 self.behaviors_parsed에 저장합니다.<br/>
   즉 데이터를 하나 뽑으면 `dict[str, Tensor]` 형태의 구조를 갖습니다.<br/>
   그런데 mini batch 학습을 위해서는 여러개의 데이터를 하나로 묶어서 batch data를 생성해야 하고, 이 기능을 `DataLoader`로 수행합니다.<br/>
   `DataLoader`에서는 여러 데이터를 하나로 묶어 `dict[str, list[Tensor]]` 형태로 변경합니다.<br/>
   `list[Tensor]`의 길이는 config파일의 batch_size로 정해집니다.<br/>
   여기서 1 epoch의 총 iteration은 behaviors의 총 데이터 수 / batch_size로, 배치 데이터의 총 개수와 같습니다.<br/>
5. 모든 모델 클래스가 상속 받는 `pl.LightningModule`의 구현 방식으로 인해, 모델의 인스턴스 자체를 함수처럼 사용하면 해당 클래스에 구현된 forward() 함수가 실행됩니다.
6. forward 함수는 학습 시 사용하는 batch data를 받아, behaviors의 사용자 history를 기반으로 해당 사용자의 impression 목록의 click probability를 예측하고 반환합니다.<br/>
   -> 상세: behaviors의 모든 데이터는 크게 유저의 history, 해당 유저의 impressions 데이터로 구성됩니다.<br/>
   여기서 history는 해당 유저가 과거에 열람한 뉴스 목록, impressions는 이러한 history를 가진 유저에게 특정 시점에 화면에 노출된 뉴스 목록입니다.<br/>
   여기서 impressions에는 유저가 해당 뉴스를 클릭했는지(1), 하지 않았는지(0)가 1과 0으로 라벨링 되어있습니다.<br/>
   즉 모델이 history만으로 impressions의 모든 뉴스 목록에 대해 해당 history를 가진 유저의 클릭 가능성을 예측하고, 라벨과 비교하거나 순위를 매겨보면 해당 모델이 추천을 얼마나 정확하게 하는지를 계산할 수 있습니다. 
7. 여기서 반환 형태는 Tensor인데, 내부 데이터는 list[list[float]] 형태입니다. 즉 입력한 모든 batch data에 대한 예측 결과가 반환되는 것입니다.<br/>
   -> 상세: 예를 들어 batch_size가 2라면, 각 배치마다 behaviors의 데이터가 2개씩 포함될 것입니다.<br/>
    따라서 예측해야할 유저와 impression 쌍도 두개이므로, 반환하는 결과 데이터도 2개입니다.

In [1]:
# jupyter notebook에서 import 해서 쓰는 모듈의 코드가 변경될 시, 변동 사항을 자동으로 반영해주는 기능 켜기
%load_ext autoreload
%autoreload 2

## 1. ckpt 파일로 모델 불러오기

In [2]:
import os
from os import path
import sys

PROJECT_DIR = path.abspath(path.join(os.getcwd(), "..", ".."))
sys.path.append(PROJECT_DIR)

from utils.model_manager import ModelManager
from utils.base_manager import ManagerArgs

c:\Users\USER\anaconda3\envs\newsrec\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
args = ManagerArgs(
    config_path = path.join(PROJECT_DIR, "config/model/nrms/exp_demo1.yaml"),
    test_ckpt_path = path.join(PROJECT_DIR, "logs/lightning_logs/checkpoints/nrms/exp_demo1/epoch=24-val_auc_epoch=0.6996.ckpt")
)

In [4]:
model_manager = ModelManager(PROJECT_DIR, args, "test")
model_manager.model.eval()
print(end="") # .eval() 반환 출력하지 않도록

Seed set to 1234
100%|██████████| 7538/7538 [00:06<00:00, 1182.94it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


## 2. 테스트용 데이터 불러오기

In [5]:
"""
모든 데이터의 첫 번째 차원의 shape 값은 batch size입니다.
즉 batch data 안에 실제로 어떤 데이터가 저장되어있는지 알아보기 위해 출력해볼 때
해당 값은 별로 의미가 없습니다. 

예를 들어 h_title의 shape 출력 결과의 각 숫자는 다음과 같은 의미를 지닙니다.
(batch_size, config파일에 설정한 max_history 값, 전처리 과정에서 설정한 max_title 값 = 제목의 최대 토큰 개수)

c_abstract은 다음과 같습니다.
(batch_size, 해당 impressions 데이터에 포함된 뉴스 개수, 전처리 과정에서 설정한 max_abstract 값 = 본문 요약의 최대 토큰 개수)
"""

batch_data = model_manager.get_batch_from_dataloader(0)
model_manager.show_batch_struct(batch_data)

<class 'dict'>
{
	user:	type=Tensor, shape=(1,), inner_type=int
	h_title:	type=Tensor, shape=(1, 50, 20), inner_type=list[list[int]]
	h_abstract:	type=Tensor, shape=(1, 50, 50), inner_type=list[list[int]]
	h_category:	type=Tensor, shape=(1, 50), inner_type=list[int]
	h_subcategory:	type=Tensor, shape=(1, 50), inner_type=list[int]
	h_vader_sentiment:	type=Tensor, shape=(1, 50), inner_type=list[float]
	h_bert_sentiment:	type=Tensor, shape=(1, 50), inner_type=list[float]
	history_length:	type=Tensor, shape=(1,), inner_type=int
	c_title:	type=Tensor, shape=(1, 28, 20), inner_type=list[list[int]]
	c_abstract:	type=Tensor, shape=(1, 28, 50), inner_type=list[list[int]]
	c_category:	type=Tensor, shape=(1, 28), inner_type=list[int]
	c_subcategory:	type=Tensor, shape=(1, 28), inner_type=list[int]
	c_vader_sentiment:	type=Tensor, shape=(1, 28), inner_type=list[float]
	c_bert_sentiment:	type=Tensor, shape=(1, 28), inner_type=list[float]
	labels:	type=Tensor, shape=(1, 28), inner_type=list[int]
}


## 3. 모델에 batch data 입력하고 출력 확인하기

In [12]:
"""
index를 바꿔서 원하는 데이터를 테스트해볼 수 있습니다.
"""
result = model_manager.show_result(0)

Rank    Score    Label  index 
--------------------------------
1      21.96449    0      8   
2      21.63985    0      21  
3      14.37056    0      20  
4      13.39139    0      22  
5      7.60424     0      17  
6      4.49830     0      26  
7      3.25125     0      23  
8      2.46181     0      0   
9      0.83084     0      1   
10     0.31242     0      11  
11     0.16820     0      16  
12     -0.67941    0      27  
13     -3.86770    1      9   
14     -5.43905    0      25  
15     -6.06323    0      18  
16     -8.16474    0      24  
17     -8.50863    0      5   
18     -8.97648    0      10  
19     -9.31507    0      6   
20    -10.63651    0      15  
21    -11.07256    0      14  
22    -12.26732    0      13  
23    -12.92857    0      3   
24    -12.96733    0      12  
25    -13.71577    0      19  
26    -17.14813    0      2   
27    -24.28173    0      7   
28    -29.99214    0      4   


In [9]:
print(result.data.shape)
print(result.shape)
print(result.tolist())

torch.Size([1, 28])
torch.Size([1, 28])
[[2.461810350418091, 0.8308407068252563, -17.148134231567383, -12.928565979003906, -29.992137908935547, -8.508628845214844, -9.315069198608398, -24.28173065185547, 21.964494705200195, -3.8677029609680176, -8.976483345031738, 0.3124246597290039, -12.967333793640137, -12.267321586608887, -11.072564125061035, -10.636512756347656, 0.16820302605628967, 7.604235649108887, -6.063232421875, -13.715771675109863, 14.370555877685547, 21.639850616455078, 13.391390800476074, 3.251248598098755, -8.164743423461914, -5.439045429229736, 4.498304843902588, -0.6794055700302124]]


In [31]:
print(batch_data['h_title'][0][20])

tensor([7264,   24, 4383,  893, 1741,  194,   22, 2301,  151, 5485,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0], device='cuda:0')


해야하는 것: 모델 출력 뽑아보기
구체적으로: 모델에 넣어준 뉴스 목록 중 가장 클릭 점수가 높은 뉴스가 어떤 뉴스인지 카테고리, 제목, 본문 요약을 출력한다.
어떻게 만들지?: 현재 batch data에 뉴스 정보가 들어가있긴 한데 일단 그거가지고 뽑아볼까?
그러려면?: word2int 데이터를 불러와아 해. 그리고 가장 예측 순위가 가장 높은 뉴스의 batch data 상에서의 인덱스도 알아야 하고.

만들어보니 문제점: 해당 프로젝트에서는 문장을 토큰 단위로 쪼개고, 고정 길이로 변환해서 어쩔 수 없이 잘리는 문장들이 존재함.
그럼 문장이 안잘리게 길이를 늘리면? -> 아마 학습속도가 느려지지 않을까.
해결책은? -> 해당 데이터의 impression ID를 알아내던가, news ID를 알아낼 수 있어야 함.
그런데 현재 batch data 구성으로는 이를 알아낼 수 없음.
그러면? -> BaseDataset에 코드를 약간 추가해서 batch data에 impression ID를 포함하도록 변경한다?
 혹은 news2int 파일을 생성해서 뉴스 ID 인덱스를 batch data에 포함하도록 코드를 짠다? -> newsID 찾기는 쉬우나 상당히 번잡스러워 보임.

## 1. word2int 가져오기

In [51]:
word2int = model_manager.get_word2int()
word2int.set_index('word_index', inplace=True)
word2int.head()

,word
word_index,
1,the
2,brands
3,queen
4,elizabeth
5,","


In [49]:
word2int.loc[1, 'word']

'the'

## 2. batch_data에서 특정 뉴스의 title, abstract 뽑아 보기

In [34]:
list(batch_data.keys())

['user',
 'h_title',
 'h_abstract',
 'h_category',
 'h_subcategory',
 'h_vader_sentiment',
 'h_bert_sentiment',
 'history_length',
 'c_title',
 'c_abstract',
 'c_category',
 'c_subcategory',
 'c_vader_sentiment',
 'c_bert_sentiment',
 'labels']

In [35]:
# 1. history의 맨 마지막 뉴스
h_title = batch_data['h_title'][0]
title: list = h_title[49].tolist()
print(title)

[7613, 204, 1136, 104, 7096, 273, 131, 310, 1104, 801, 372, 1807, 46, 29, 12290, 24, 1534, 1115, 0, 0]


In [55]:
def get_str_from_index_list(word2int, index_list):
    words = []
    for index in index_list:
        if index == 0:
            continue
        word = word2int.loc[index]['word']
        words.append(word)

    sentence = ' '.join(words)
    return sentence

sentence = get_str_from_index_list(word2int, title)
print(sentence)

bridge has definitely been burned ' : williams says redskins have smeared him in aftermath of cancer diagnosis
